In [9]:
import librosa 
import librosa.display
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from tensorflow import keras 
from tensorflow.keras.preprocessing import image_dataset_from_directory, image 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight

In [3]:
path = Path.cwd()/'data'
answers = pd.read_csv('answers.csv')
images_directory = path/'train_images'

In [14]:
cb_training = image_dataset_from_directory(images_directory, labels='inferred', image_size=(97,97), subset='training', validation_split=.2, seed=10)
cb_validation = image_dataset_from_directory(images_directory, labels='inferred', image_size=(97,97), subset='validation', validation_split=.2, seed=10)

Found 30000 files belonging to 2 classes.
Using 24000 files for training.
Found 30000 files belonging to 2 classes.
Using 6000 files for validation.


# Transfer  

## Bunches of Metrics

In [6]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]


## Class Weight Computation

In [7]:

xx =compute_class_weight(class_weight='balanced',classes=np.unique(answers.label), y=answers.label)
class_weight = dict(zip(np.unique(answers.label), xx))

In [17]:
earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
checkpoint = ModelCheckpoint('transfer_vgg19_default_learn_rate.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')



## Model VGG19 Transfer Learning 1 - Default Learn Rate

In [20]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(97,97,3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)

x = Dense(1000, activation='relu')(x)
x = Dropout(.5)(x)
x = Dense(1000, activation='relu')(x)
x = Dropout(.5)(x)


predictions = Dense(1, activation='sigmoid')(x)

model =  Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=METRICS)

In [21]:
history = model.fit(cb_training, epochs=10,validation_data=cb_validation, class_weight = class_weight, callbacks=[earlystop,checkpoint])

Epoch 1/10
750/750 [==============================] - 758s 1s/step - loss: 1.1447 - tp: 9445.0000 - fp: 16918.0000 - tn: 19169.0000 - fn: 1572.0000 - accuracy: 0.6075 - precision: 0.3583 - recall: 0.8573 - auc: 0.7638 - prc: 0.4648 - val_loss: 0.3378 - val_tp: 1175.0000 - val_fp: 737.0000 - val_tn: 3864.0000 - val_fn: 224.0000 - val_accuracy: 0.8398 - val_precision: 0.6145 - val_recall: 0.8399 - val_auc: 0.9093 - val_prc: 0.7493

Epoch 00001: val_loss improved from inf to 0.33776, saving model to transfer_vgg19_default_learn_rate.h5
